# k-Means Clustering

The k-means clustering algorithm is one of the most intuitive unsupervised learning algorithm. It's goal is to deduce natural clusters (also called "classes") in a dataset by figuring out which data points are most similar to each other. It's called "k-means" because it results in $k$ classes using a method that attempts to find the mean of each class of data points.

## How It Works

Many machine learning methods work via a similar pattern: (1) they make a guess about the answer, (2) they make a small change to the guess that improves its goodness slightly, then (3) they repeat step 2 until the guess is "good enough" in some way. The k-means algorithm is a great example of this approach. It works as follows.

### K-Means Algorithm

#### Inputs
1. **A set of points**. There may be any number of points, and they may have any number of dimensions (columns). All the dimensions should be numerical.
2. **$k$**. The number of clusters to find.

#### Algorithm
1. **Guess**. The algorithm randomly chooses $k$ points. These "cluster points" will have the same dimensionality as the input points.
2. **Iterate**. These steps are performed repeatedly until some threshold is reached.
   1. **Assign clusters**. For each point in the input, the algorithm figures out which of the $k$ cluster points is closest to it and assigns it that cluster value.
   2. **Update clusters**. For each cluster, update the position of the cluster point to be the mean of all input points that belong to the cluster.

#### Outputs
1. The $k$ cluster points representing the $k$ clusters found by the method; each data point is the mean position of all the data points in one of the $k$ clusters.
2. An assignment of all the input points into one of the $k$ clusters.

### Demonstration

The following animation demonstrates how the k-means algorithm works generally.

In this animation, the black dots are the data points that we are hoping to cluster based on their `(x,y)` position. Each star represents one of the three clusters.

Initially, the three stars have a random position that doesn't do a very good job of representing where the underlying clusters are. However, after a few iterations of the k-means algorithm, in which we repeatedly assign each data point to its nearest cluster then move each cluster's position to the mean position of its assigned data points, we reach a good clustering.

In [ ]:
# This code-block generates and displays an animation of the K-Means algorithm
# for a random set of points.
# This specific code uses the `matplotlib.animation.FuncAnimation` class to
# generate the animation. A very simple implementation of the k-means
# clustering algorithm can be found in the `run_kmeans` function below, but
# understanding this code isn't important for the K-Means lesson.

def _make_kmeans_animation(seed=5, n=100):
    import numpy as np
    from scipy.spatial import KDTree
    import ipywidgets as ipw
    import matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    from IPython.display import HTML
    
    # The random seed:
    np.random.seed(seed)
    # Number of data points:
    #n = 100
    # Number of clusters:
    k = 3
    # The number of steps:
    s = 6
    # x/y min and max:
    (xmin,xmax) = (-10, 10)
    (ymin,ymax) = (-10, 10)
    # The colors for each cluster:
    clust_colors = np.array([(1, 0.2, 0.2), (0, 0.8, 0.8), (0.7, 0.7, 0.7)])
    
    def random_datapoints_clustered(center=None, scale=None,
                                    n=n, out=None,
                                    xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax):
        # Draw a random center:
        if center is None:
            center = np.random.rand(2) * [xmax - xmin, ymax - ymin] + [xmin, ymin]
        else:
            center = np.array(center)
        # Pick a random scale:
        if scale is None:
            scale = np.random.exponential(2.0)
        # Draw some angles and radii.
        th = np.random.rand(n) * (2*np.pi)
        r = np.random.randn(n) * scale
        # Convert these to (x,y):
        xy = center[:,None] + [r*np.cos(th), r*np.sin(th)]
        # Ready the output:
        if out is None:
            out = np.empty_like(xy)
        # Make sure none of the xy are out of range:
        return np.stack(
            [np.mod(xy[0] - xmin, xmax - xmin) + xmin,
             np.mod(xy[1] - ymin, ymax - ymin) + ymin],
            axis=0,
            out=out)
    def random_datapoints(centers=None, scales=None,
                          n=n, k=k, out=None,
                          xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax):
        if out is None:
            out = np.empty((2, n), dtype=float)
        # We make k clusters...
        step = n // k
        for clustno in range(k - 1):
            random_datapoints_clustered(
                center=(None if centers is None else centers[clustno]),
                scale=(None if scales is None else scales[clustno]),
                n=step,
                out=out[:,(clustno*step):((clustno+1)*step)],
                xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
        random_datapoints_clustered(
            center=(None if centers is None else centers[k-1]),
            scale=(None if scales is None else scales[k-1]),
            n=(out.shape[1] - (k-1)*step), 
            out=out[:,((k-1)*step):],
            xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
        return out
    def random_clusterpoints(k=k, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax):
        return np.random.rand(2,k) * [[xmax-xmin],[ymax-ymin]] + [[xmin],[ymin]]
    
    # Also make the data and cluster points we will use:
    coords = random_datapoints([(8,4), (2,-6), (-6,-4)], [2, 2, 2], n=n)
    clust0 = random_clusterpoints()
    clusts = np.empty_like(clust0, shape=((s + 1,) + clust0.shape))
    labels = np.zeros((s + 1, n), dtype=np.uint8)
    # Actually perform the k-means clustering steps:
    def run_kmeans(clust0, n=n, k=k, s=s,
                   coords=coords, clusts=clusts, labels=labels):
        clusts[0] = clust0
        for stepno in range(s):
            near = KDTree(clusts[stepno].T)
            (d,ii) = near.query(coords.T)
            labels[stepno] = ii
            clusts[stepno + 1] = np.stack(
                [np.mean(coords[:, labels[stepno] == ll], axis=1)
                 for ll in range(k)],
                axis=1)
        near = KDTree(clusts[s].T)
        (d,ii) = near.query(coords.T)
        labels[s] = ii
    run_kmeans(clust0)
    
    # Make the figure we'll use:
    (fig,ax) = plt.subplots(1,1, figsize=(3, 3), dpi=180)
    fig.subplots_adjust(0,0,1,1,0,0)
    ax.set_xlim([xmin-0.5, xmax+0.5])
    ax.set_ylim([ymin-0.5, ymax+0.5])
    ax.plot([xmin, xmax], [0, 0], 'k:', lw=0.25, zorder=-1)
    ax.plot([0, 0], [ymin, ymax], 'k:', lw=0.25, zorder=-1)
    ax.axis('off')
    # Draw a few things we'll use in all the figure frames:
    coordplot = ax.scatter(
        coords[0], coords[1], 
        fc='k', s=8, lw=0.5, edgecolor=None)
    clustplot = ax.scatter(
        clusts[0,0], clusts[0,1],
        fc=clust_colors, s=50, lw=0.5, ec='k', marker='*')
    arrowplots = [
        ax.arrow(
            clusts[0,0,ii], clusts[0,1,ii], 1, 1,
            lw=0.5, color=clr, head_width=0.1)
        for (ii,clr) in enumerate(clust_colors)]
    textplot = ax.text(
        -9.5, 7.5, "Round",
        fontsize=10, horizontalalignment='left', verticalalignment='top')
    
    # We can now define how to draw a frame; there are actually three kinds of
    # frames: (1) basic plots, (2) cluster-step plots, and (3) update-step plots.
    def basic_frameplot(stepno,
                        coords=coords, clusts=clusts,
                        fig=fig, ax=ax, clustplot=clustplot,
                        xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax):
        x = clusts[stepno, 0]
        y = clusts[stepno, 1]
        clustplot.set_offsets(np.c_[x,y])
        for arrow in arrowplots:
            arrow.set_visible(False)
        coordplot.set_edgecolor(None)
        textplot.set_text(f"Round {stepno}: Start")
    def cluster_frameplot(stepno,
                          coords=coords, clusts=clusts,
                          fig=fig, ax=ax, clustplot=clustplot,
                          xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax):
        x = clusts[stepno, 0]
        y = clusts[stepno, 1]
        clustplot.set_offsets(np.c_[x,y])
        lbl = labels[stepno]
        coordplot.set_edgecolor(clust_colors[lbl])
        for arrow in arrowplots:
            arrow.set_visible(False)
        textplot.set_text(f'Round {stepno}: Assign Clusters')
    def update_frameplot(stepno,
                         coords=coords, clusts=clusts,
                         fig=fig, ax=ax, clustplot=clustplot,
                         xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax):
        (x0, y0) = clusts[stepno]
        (x, y) = clusts[stepno + 1]
        (dx, dy) = (x - x0, y - y0)
        for (ii,arrow) in enumerate(arrowplots):
            arrow.set_data(x=x0[ii], y=y0[ii], dx=dx[ii], dy=dy[ii])
            arrow.set_visible(True)
        coordplot.set_edgecolor(clust_colors[labels[stepno]])
        textplot.set_text(f"Round {stepno}: Update Cluster Means")
    # Draw the step 0 frame:
    def draw_frame0(coords=coords, clusts=clusts,
                    fig=fig, ax=ax, clustplot=clustplot,
                    xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax):
        basic_frameplot(
            0,
            coords=coords, clusts=clusts,
            fig=fig, ax=ax, clustplot=clustplot,
            xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
        textplot.set_text("Initial State")
    def frameplot(stepno,
                  coords=coords, clusts=clusts,
                  fig=fig, ax=ax,
                  xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax):
        opts = dict(
            coords=coords, clusts=clusts,
            fig=fig, ax=ax,
            xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
        if stepno == 0:
            draw_frame0(**opts)
        elif stepno == 9:
            cluster_frameplot(4, **opts)
            textplot.set_text('Final Clusters')
        elif stepno % 3 == 0:
            basic_frameplot(stepno // 3, **opts)
        elif stepno % 3 == 1:
            cluster_frameplot(stepno // 3, **opts)
        else:
            update_frameplot(stepno // 3, **opts)
        return [textplot, coordplot, clustplot] + arrowplots
    
    anim = FuncAnimation(
        fig,
        frameplot,
        frames = 10,
        interval = 1500,
        blit = True)
    plt.close(fig)
    display(HTML(anim.to_jshtml()))

# Call the above function, which displays the javascript animation.
_make_kmeans_animation()

## Example: Finding Clusters in the California Housing Dataset

Let's work through an example of k-means clustering with a real dataset. We'll use the California Housing Dataset that we examined earlier in the [Introduction to Unsupervised Learning](0_introduction), and we'll attempt to find geographical clusters of census parcels using k-means clustering.

### Load the Dataset

In [ ]:
# First, we need to import scikit-learn:
import sklearn as skl

# Next, we use scikit-learn to download and return the CA housing dataset:
ca_housing_dataset = skl.datasets.fetch_california_housing()

# Extract the actual data rows and the feature names:
ca_housing_data = ca_housing_dataset['data']
ca_housing_featnames = ca_housing_dataset['feature_names']

# We also extract the target data:
ca_housing_targdata = ca_housing_dataset['target']
ca_housing_targnames = ca_housing_dataset['target_names']

# To organize the dataset into a dataframe, we use Pandas:
import pandas as pd

feat_df = pd.DataFrame(
    {k: v for (k,v) in zip(ca_housing_featnames, ca_housing_data.T)})
targ_df = pd.DataFrame({ca_housing_targnames[0]: ca_housing_targdata})

# Display the feature dataframe:
feat_df

### Visualize the Longitude and Latitude Data

We can start by visualizing the raw data that we're going to use for clustering. We're using the longitude and latitude of each census parcel in the dataset, so a visualization of these coordinates should be resemble the state of California.

In [ ]:
# Extract the longitude and latitude from the dataset features.
x = feat_df['Longitude']
y = feat_df['Latitude']

# We'll need matplotlib to make the plots:
import matplotlib.pyplot as plt
# Make a quick scatter-plot where each dot is a parcel center.
plt.scatter(x, y, c='k', s=0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
# Make sure the plot axis represents x and y space equally.
plt.axis('equal')
# Show the plot.
plt.show()

Clearly there are clusters of parcels near the most populated parts of the state: Los Angeles, San Diego, San Francisco and the Bay Area, and Sacramento in particular. While there is no one correct way to cluster the parcels in this map, we might assess that a method of clustering performs well if it tends to identify these areas as distinct clusters.

Let's see how to use k-means for this task and how well it performs.

In [ ]:
import numpy as np

# Make an (Nx2) coordinate matrix of the latitude and longitude:
coords = np.c_[x,y]
# Use scikit-learn's KMeans algorithm; we'll search for 8 clusters:
n_clusters = 8
kmeans = skl.cluster.KMeans(n_clusters=n_clusters)
kmeans.fit(coords)

Once the above cell has been run, the `kmean` object has been fitted, meaning that it has found cluster centers and cluster labels for the parcels in the census. We can access these discovered values as fields of the `kmeans` object itself.

In scikit-learn, this paradigm is very common: one creates an object (i.e., a `sklearn.cluster.KMeans` object) that takes some parameters (the number of clusters), then you provide the object with data via a `fit` function. After calling the fitting routine, the object has been trained and its methods and fields can be queried.

The fields that scikit-learn provides for the user typically end with an underscore (`_`) character.

In [ ]:
# This gives us one label per parcel in the census.
label = kmeans.labels_
# And one cluster center per cluster.
centers = kmeans.cluster_centers_

# Make another scatter plot of the parcels, but color them by their cluster
# label this time:
plt.scatter(x, y, c=label, s=0.5, cmap='hsv')
# Also plot the cluster centers as stars.
plt.scatter(
    centers[:,0], centers[:,1],
    edgecolor='k',
    c=np.linspace(0,1,n_clusters),
    cmap='hsv', 
    s=50,
    marker='*')
# Fix up the plot to use isomorphic pixel sizes.
plt.axis('equal')
# And plot the figure.
plt.show()

Clearly, the algorithm found some clusters that make sense, but it's simultaneously hard to say whether these clusters are "correct" in any meaningful sense. In part, this is because geographic clustering is a complicated topic&mdash;politics, culture, geography, and population all play major roles in our societal notion of a geographical regions. However, k-means is also limited in that it cares only about a simple metric of closeness, and it's simply not always the case that spatial closeness is a good indicator of whether two locations are part of a geographical cluster.

Note additionally that k-means is a stochastic algorithm, meaning that if you run it several times, you will likely get slightly different results each time. If you run the two cells above several times, you can get a sense for the kinds of clusterings k-means can produce from similar data. 

**After running the cell above several times, what are some of the strengths and weaknesses of this method that you observe?**

```{dropdown} Some possible strengths...

* **K-means runs quickly**. In fact, you can run it many times even for fairly large datasets.
* **K-means classifies all points**. There is never an outlier point that the k-means algorithm misses or fails to classify.
* **K-means is easy to understand**. The k-means method itself is straightforward, and others will have little trouble understanding results based on k-means.
```

```{dropdown} Some possible weaknesses...

* **K-means is not especially reliable**. The answer you get differs quite a bit across different runs of k-means, at least for this problem.
* **K-means can't figure out the number of clusters**. Often we won't know the correct number of clusters and would rather the algorithm figure it out for us.
* **K-means does not care about density**. It does not try to draw cluster boundaries where the density is lowest.
```

## Additional Resources

* [k-means at Wikipedia](https://en.wikipedia.org/wiki/K-means_clustering)
* [k-means user-guide at Scikit-learn](https://scikit-learn.org/stable/modules/clustering.html#k-means)
* [k-means documentation at Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)